In [ ]:
import os
import json
import pandas as pd
import glob

UPGRADE_HEIGHT=4707300
HALT_HEIGHT=4713064

## Parse pool data by Geo

To downlod the data execute the script `data/pools/download_geo_pool_data.sh`

The data was parsed from the blockchain with the following code: https://github.com/george-aj/osmosis-nitrogen-extra-gamm-analysis/blob/main/shadow_pool_estimates/shadow_pool_model.py#L126-L150

In [ ]:
# Data downloaded by geo
DATA_FOLDER="data/pools/geo/"
results = []

pool_data_at_height_files = glob.glob(os.path.join(DATA_FOLDER, '*.json'))

for file in pool_data_at_height_files:

    print("Processing ", file, end='\r')

    with open(file) as f:
        data = json.load(f)

        # Get height from file path
        height = file.replace(DATA_FOLDER, '').replace(".json", "")

        # Create a row for each pool and add the height as a column
        for pool_data in data.values():
            pool_data["height"] = height
            results.append(pool_data)

In [ ]:
# Import data to dataframe
df = pd.json_normalize(results)
df.head()

In [ ]:
# Expand `poolAssets`
df = df.join(pd.json_normalize(pd.json_normalize(df["poolAssets"])[0])).rename(columns = {'token.denom':'token0.denom', 'token.amount':'token0.amount', "weight": "token0.weight"})
df = df.join(pd.json_normalize(pd.json_normalize(df["poolAssets"])[1])).rename(columns = {'token.denom':'token1.denom', 'token.amount':'token1.amount', "weight": "token1.weight"})
df.drop(columns=["poolAssets"], inplace=True)

df.head()

In [ ]:
# Reorder columns
df = df[[
 'id', 
 'height',
 'totalWeight',
 'totalShares.denom', 
 'totalShares.amount',  
 'token0.weight',
 'token0.denom',
 'token0.amount',
 'token1.weight',
 'token1.denom',
 'token1.amount',
 'address',
 '@type',
 'futurePoolGovernor',
 'poolParams.swapFee',
 'poolParams.exitFee',
 ]]

# Sort values
df.sort_values(by=['id', 'height'], inplace=True)

# Save to file
df.to_csv("csv/pools/geo/all_pools.csv", index=None)

## Parse pool data by osmosis-data

To downlod the data execute the script `data/pools/download_osmosis_pool_data.sh`

In [ ]:
# Data downloaded by osmosis-data
DATA_FOLDER="data/pools/osmosis/"
results = []

pool_data_at_height_files = glob.glob(os.path.join(DATA_FOLDER, '*.json'))

for file in pool_data_at_height_files:

    print("Processing ", file, end='\r')

    with open(file) as f:
        data = json.load(f)

        # Get height from file path
        height = file.replace(DATA_FOLDER, '').replace(".json", "")

        # Create a row for each pool and add the height as a column
        for pool_data in data:
            pool_data["height"] = height
            results.append(pool_data)

### Create a DataFrame with all the data

In [ ]:
# Import to dataframe
df = pd.json_normalize(results)
df.head()

In [ ]:
# Expand `poolAssets`
df = df.join(pd.json_normalize(pd.json_normalize(df["poolAssets"])[0])).rename(columns = {'token.denom':'token0.denom', 'token.amount':'token0.amount', "weight": "token0.weight"})
df = df.join(pd.json_normalize(pd.json_normalize(df["poolAssets"])[1])).rename(columns = {'token.denom':'token1.denom', 'token.amount':'token1.amount', "weight": "token1.weight"})
df.drop(columns=["poolAssets"], inplace=True)

In [ ]:
# Reorder columns
df = df[[
 'id', 
 'height',
 'totalWeight',
 'totalShares.denom', 
 'totalShares.amount',  
 'token0.weight',
 'token0.denom',
 'token0.amount',
 'token1.weight',
 'token1.denom',
 'token1.amount',
 'address',
 '@type',
 'futurePoolGovernor',
 'poolParams.swapFee',
 'poolParams.exitFee',
 ]]

# Sort values
df.sort_values(by=['id', 'height'], inplace=True)

# Save to file
df.to_csv("csv/pools/osmosis/all_pools.csv", index=None)